## [Python Programming for Biologists, Tel-Aviv University / 0411-3122 / Spring 2016](http://py4life.github.io/TAU2016/)
# Solution for homework 5
### General instructions
- In this excercise, you will have to read and parse files. Download them [here](https://github.com/Py4Life/TAU2016/raw/master/files_for_hw5.rar), extract the rar file and then save the directory to your course directory (or wherever you access the notebooks from).
- You will have to use the _Comma Separated Values_ (CSV) format, and the python csv module. To learn how to use it, go over the appropriate section in lecture 4. You can also learn more [here](https://docs.python.org/3/library/csv.html).

## 1) GATA4 promoters
In class, we dealt with the binding site of GATA4 TF.
Now, let's extract some statistics from the promoters of GATA4.

We'll use the check_for_GATA4() function from class, denoted below (run it to compile it).

In [1]:
import re
def check_for_GATA4(sequence):
    motif_regex = re.compile(r'AGATA[AG][AC]AG[AG][CG]A')
    if re.search(motif_regex,sequence):
        return True
    else:
        return False
    

The 'GATA4_promoters.fasta' file in the attached rar archive includes (made-up) promoter sequences for genes suspected to be regulated by GATA-4. (Open it with notepad to see the format!). 
We'll use everything we've learned so far to write a program that summarizes some interesting statistics regarding the GATA-4 motifs in these promoters.  
First, let's adjust the parse\_fasta() function we created in class for the specific format of the promoters file:

In [2]:
def parse_promoters_fasta(file_name):
    """
    Receives a path to a fasta file, and returns a dictionary where the keys
    are the sequences names and the values are the sequences.
    """
    # create an empty dictionary to store the sequences
    sequences = {}
    # open fasta file for reading
    with open(file_name,'r') as f:
        # Loop over file lines
        for line in f:
            # if header line
            if line.startswith('>'):
                seq_id = line[1:-1]   # take the whole line, except the '>' in the beginning and '\n' at the end
            # if sequence line
            else:
                seq = line.strip()
                sequences[seq_id] = seq
    return sequences

1)
Write a function that receives a promoters fasta dictionary, and counts how many of the promoters have the GATA-4 motif. Use any of the functions defined above and complete the code:

In [3]:
def count_promoters_with_motif(promoters_dictionary):
    """
    Receives a dictionary representing a promoters fasta file,
    and counts how many of the promoters include a GATA-4 motif.
    """
    promoters_count = 0   # store the number of promoters with GATA-4 motif
    # write your code here
    for k in promoters_dictionary:
        sequence = promoters_dictionary[k]
        if check_for_GATA4(sequence):
            promoters_count += 1 # a short way to write: promoters_count = promoters_count + 1
    
    return promoters_count
    

#Test your code:
promoters_dict = {"prom1": "CCGGGAAAAGGGTCCCACCTGACGGGAAATTGGAGTGGAGGGCGACAAATCATCTAGTTTAAGTTCGGCCTGTCACACATTGTACATGAGATAAAAGGCA",
                 "prom2": "ACCCTGGTCTGAACTAGTGAATTTCCTTCTACGTACAGAGTCGACAACGAGCCCGTAGGGCTTGTCATCTCAAGATAGCAGAGAAGCGTGAACCGCTTCG",
                 "prom3": "TGCGGGACATGTCACAGATAACAGAGATCGAAGCTGCATCCGTATGTTCGTTCGGGCACATTGTGAAAGACATCAACGTACTTCGCCCTTTGGCAGGCGA"}
gata4_cnt = count_promoters_with_motif(promoters_dict)
print(gata4_cnt, "of the promoters in promoter_dict have GATA-4 motif!")

3 of the promoters in promoter_dict have GATA-4 motif!


2) For promoters that do include the GATA-4 motif, we would like to know the frequencies of the different nucleotides for each of the four variable positions in the motif. Complete the code:

In [4]:
def get_positions_statistics(promoters_dictionary):
    """
    Receives a dictionary representing a promoters fasta file,
    and returns the frequencies of possible nucleotides in 
    each variable position.
    """
    # define a  dictionary for each position, to store the nucleotide frequencies
    # D position
    D_dict = {'A':0, 'G':0, 'T':0}
    # M position
    M_dict = {'A':0, 'C':0}
    # R position
    R_dict = {'A':0, 'G':0}
    # S position
    S_dict = {'C':0, 'G':0}
    
    # iterate over promoters
    for p in promoters_dictionary: #iterates over the keys ==> p is a different key in every iteration!
        # if promoter includes the GATA-4 motif
        sequence = promoters_dictionary[p]
        if check_for_GATA4(sequence):
            # get variable nucleotides in promoter
            motif_regex = re.compile(r'AGATA([AG])([AC])AG([AG])([CG])A') # notice the parentheses
            result = re.search(motif_regex,sequence)
            # insert to dictionaries
            D = result.group(1)
            D_dict[D] += 1
            M = result.group(2)
            M_dict[M] += 1
            R = result.group(3)
            R_dict[R] += 1
            S = result.group(4)
            S_dict[S] += 1
            
    return D_dict, M_dict, R_dict, S_dict

3) Now, we just have to write a function that will summarize the results in a CSV file. It should receive the frequencies dictionaries and write statistics to an output file. Complete the code:

In [6]:
def summarize_results(D_dict, M_dict, R_dict, S_dict, output_file):
    with open(output_file, 'w') as fo: #first, we open a file writer with open()
        csv_writer = csv.writer(fo) #Now, wrap it with csv writer
        # write headers line
        csv_writer.writerow(['Position','A','G','C','T'])
        # summarize D position
        csv_writer.writerow(['D',D_dict['A'],D_dict['G'],0,D_dict['T']])
        # summarize M position
        csv_writer.writerow(['M',M_dict['A'],0,M_dict['C'],0])
        # summarize R position
        csv_writer.writerow(['R',R_dict['A'],R_dict['G'],0,0])
        # summarize S position
        csv_writer.writerow(['S',0,S_dict['G'],S_dict['C'],0])

4) Now that we have all the functions ready, we can write the main program. Check your output!
__Make sure that the files are in the directory files_for_hw5 from where you run this notebook!__

In [9]:
import csv
promoters_file = "files_for_hw5/GATA4_promoters.fasta"
output_file = "files_for_hw5/promoters_stats.csv"

# parse fasta file
promoters_dict = parse_promoters_fasta(promoters_file)

# Count promoters with/without GATA-4 motif
promoters_with_motif = count_promoters_with_motif(promoters_dict)
promoters_without_motif = len(promoters_dict) - promoters_with_motif
print('Total promoters:',promoters_with_motif + promoters_without_motif)
print('Promoters with GATA-4 motif:',promoters_with_motif)
print('Promoters without GATA-4 motif:',promoters_without_motif)

# Get statistics
D_dict, M_dict, R_dict, S_dict = get_positions_statistics(promoters_dict)
# write to CSV
summarize_results(D_dict, M_dict, R_dict, S_dict,output_file)

Total promoters: 104
Promoters with GATA-4 motif: 58
Promoters without GATA-4 motif: 46


## 2) Plant names
A full scientific name of a plant species consists of a genus name, a species name and an authority (usually a short for the name of the person to first describe the species). For example, in _Arabidopsis thaliana (L.) Heynh._, _Arabidopsis_ is the genus, _thaliana)_ is the species and _(L.) Heynh._ is the authority. 

- The genus always begins with a capital letter, followed by one or more lower case letters. This can be found with a regex '[A-Z][a-z]+'.
- After the genus there's a space and then the species name. A space can be simply represented by a space ' ' or with '\s'.
- The species is all lowercase. 
- After the species name there's a space and then the authority.
- The authority can include any character. 

In addition, a name may (or may not) include an infraspecific rank. This is added after the species name, and consists of an _epithet_ which is either _subsp._ (for subspecies) or _var._ (for variety). The epithet is followed by the name of the infraspecific rank. 

For example, in _Fraxinus americana var. acuminata (Lam.) K.Koch_, the genus is _Fraxinus_, the species is _americana_, the infraspecies is _var. acuminata_ and the authority is _(Lam.) K.Koch_.  

The file `files_for_hw5/plant_names.txt` contains a list of plant names. The goal is to break these names into their components. 

Write a program that reads the names from the file and writes a new CSV file with the following column names: _Genus_, _Species_, _Infraspecific_ and _Authority_. 
Each plant name in `plant_names.txt` should then be processed (use regular expressions) and its components inserted into the CSV file.

In [11]:
import csv
import re

# regex for processing plant names
name_regex = re.compile(r'^([A-Z]\w+)\s(\w+)\s((subsp\.|var\.)\s\w+\s)?(.+)$')

# files
list_file = 'files_for_hw5/plant_names.txt'
out_csv = 'files_for_hw5/parsed_plant_names.csv'

def process_plant_name(name_string):
    """
    Receives a plant name and breaks it to components.
    Returns a list: [genus, species,infraspecific (if not, empty string), authority]
    """
    match_result = re.search(name_regex,name_string)
    if match_result:    # if a match was found
        genus = match_result.group(1)
        species = match_result.group(2)
        infra = match_result.group(3)
        author = match_result.group(5)
        return [genus, species, infra, author]
    else:    # if no match
        return None


with open(list_file,'r') as f:
    with open(out_csv,'w',newline='') as fo:
        csv_writer = csv.writer(fo)
        # write csv headers
        csv_writer.writerow(['Genus','Species','Infraspecific','Authority'])
        # iterate on names and process
        for name in f:
            name = name.strip()
            name_parts = process_plant_name(name)
            if name_parts:   # if name was processed successfully
                csv_writer.writerow(name_parts)
            else:   # warn that something is wrong with the name
                print('Name',name,'could not be processed.')
print('Parsing completed')

Parsing completed


## 3) Live debugging - find genes in a sequence
Here is a working version of the code.
How do we know it's _working_? Because **the assertion succeeds**.

In [ ]:
bases = "ACGT"
start = "ATG"
stops = ["TAG","TGA","TAA"]

def is_gene(sequence):
    if len(sequence) < 6: # check minimum length 
        return False
    if len(sequence) % 3 != 0: # check length divides by 3
        return False
    if sequence[:3] != start: # check start codon
        return False
    # check stop codon
    if sequence[-3:] not in stops: 
        return False
    # check only legal characters
    for c in sequence: 
        if c not in bases:
            return False
    # check no stop codons in the middle 
    for i in range(0, len(sequence) - 6, 3): 
        if sequence[i:i+3] in stops:
            return False
    return True

def find_genes(sequence):
    start = "ATG"
    stops = ["TAG","TGA","TAA"]
    start_idx = []
    for i in range(len(sequence) - 2):
        if sequence[i:i+3] == start:
            start_idx.append(i)
    stop_idx = []
    for i in range(len(sequence) - 2):
        if sequence[i:i+3] in stops:
            stop_idx.append(i)
    genes = []
    for i in start_idx:
        for j in stop_idx:
            if j > i and (j-i)%3==0:
                gene = sequence[i:j+3]
                if is_gene(gene):
                    genes.append(gene)
    return genes

seq = 'GCCGTTTGTACTCCATTCCAATGAGGTCGCTTCATGTCAGCGAGTTTTAACGTGGTTCTTCGCTGATGTGCTGTATATGA'
genes = find_genes(seq)
assert len(genes) == 3, "Found %d genes, expected 3" % len(genes)
print("Success")